In [ ]:
from numpy import *
import numpy as np
import random
from numba import njit

In [ ]:
@njit
def init_P(num_of_passengers,grid_size=80):

  # col 1: start x
  # col 2: start y
  # col 3: fin x
  # col 4: fin y
  # col 5: distance
  # col 6: 1=passenger received ride, 0=passenger is waiting
  # col 7: passenger wait time

  P=np.random.randint(low=0, high=grid_size, size=(num_of_passengers,7))
  P[:,2:8]=0
  P[:,4]=np.random.randint(low=2,high=10,size=num_of_passengers)
  dist=np.zeros((num_of_passengers,2))

  for i in range(num_of_passengers):
    dist[i,0]=np.random.randint(low=0, high=P[i,4])
    dist[i,1]=P[i,4]-dist[i,0]
    P[i,2]=P[i,0]+dist[i,0]
    P[i,3]=P[i,1]+dist[i,1]
    if (P[i,2]>grid_size): P[i,2]=P[i,2]-grid_size
    if (P[i,2]<0): P[i,2]=grid_size+P[i,2]
    if (P[i,3]>grid_size): P[i,3]=P[i,3]-grid_size
    if (P[i,3]<0): P[i,3]=grid_size+P[i,3]
  
  return P

@njit
def init_T(num_of_taxis=6785,grid_size=80):

  # col 1: start x
  # col 2: start y
  # col 3: time out (number of periods the taxi is unavailable for pickups, advances, etc.)
  # col 4: blocks driven w/o a passenger

  T=np.random.randint(low=0, high=grid_size, size=(num_of_taxis,4))
  T[:,2:4]=0

  return T

@njit
def taxi_pickup(P,T):      
  for i in range(P.shape[0]):
    for j in range(T.shape[0]):
      if (P[i,0]==T[j,0])&(P[i,1]==T[j,1])&(P[i,5]<=0)&(T[j,2]<=0):
        # passenger riding
        P[i,5]=1
        # taxi busy
        T[j,2]=P[i,4]
        # taxi moves to the passenger's final destination
        T[j,0]=P[i,2]
        T[j,1]=P[i,3]
  
  return P,T

@njit
def empty_cabs_move(T, grid_size=80):
  num_of_free_cabs=T[T[:,2]<=0].shape[0]
  directions=np.array([[-1,0],[1,0],[0,1],[0,-1]])
  
  ## update direction
  T[np.where(T[:,2]<=0)[0],3]+=1

  ## update taxi time-out
  T[np.where(T[:,2]<=0)[0],0:2]=T[np.where(T[:,2]<=0)[0],0:2]+directions[np.random.randint(low=0, high=3, size = num_of_free_cabs),:]
  T[:,0]=np.where(T[:,0]>grid_size,T[:,0]-grid_size,T[:,0])
  T[:,0]=np.where(T[:,0]<0,grid_size+T[:,0],T[:,0])
  T[:,1]=np.where(T[:,1]>grid_size,T[:,1]-grid_size,T[:,1])
  T[:,1]=np.where(T[:,1]<0,grid_size+T[:,1],T[:,1])

  T[:,2]=T[:,2]-1
  T[:,2]=np.where(T[:,2]<0,0,T[:,2])

  return T

@njit
def step_sim(P,T,max_arrivals=100):
  
  P_new = init_P(max_arrivals)
  P = np.append(P,P_new,axis=0)

  T = empty_cabs_move(T)
  PT = taxi_pickup(P,T)
  P = PT[0]
  T = PT[1]
  
  P[np.where(P[:,5]<=0)[0],6]+=1


  return P,T

@njit
def day_sim(P,T,max_blocks=200):
  
  for i in range(max_blocks):
    arrivals=sum(np.random.poisson(0.05,80**2))
    PT=step_sim(P,T,arrivals)
    P=PT[0]
    T=PT[1]

  return P 
